# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [0]:
# try this project here!

In [12]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0831 00:20:46.347529 140175845259136 hook.py:102] Torch was already hooked... skipping hooking process


In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_wroker")

In [14]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])


W0831 00:20:51.654882 140175845259136 base.py:654] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0831 00:20:51.657563 140175845259136 base.py:654] Worker secure_wroker already exists. Replacing old worker which could cause                     unexpected behavior
W0831 00:20:51.659626 140175845259136 base.py:654] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0831 00:20:51.662590 140175845259136 base.py:654] Worker secure_wroker already exists. Replacing old worker which could cause                     unexpected behavior
W0831 00:20:51.667102 140175845259136 base.py:654] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0831 00:20:51.668879 140175845259136 base.py:654] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_wroker #objects:64>

In [0]:
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad = True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad = True)

In [0]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [0]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
model =  nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [20]:

for num_rounds in range(10):
    
    bobs_opt = optim.SGD(params = bobs_model.parameters(), lr = 0.1)
    alices_opt = optim.SGD(params = alices_model.parameters(), lr = 0.1)
    
    for i in range(10):

      bobs_opt.zero_grad()
      bobs_pred = bobs_model(bobs_data)
      bobs_loss = ((bobs_pred - bobs_target)**2).sum()


      bobs_loss.backward()
      bobs_opt.step()
      bobs_loss = bobs_loss.get().data
      bobs_loss

      alices_opt.zero_grad()
      alices_pred = alices_model(alices_data)
      alices_loss = ((alices_pred - alices_target)**2).sum()


      alices_loss.backward()
      alices_opt.step()
      alices_loss = alices_loss.get().data
      alices_loss



    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data)/2).get())
    model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data)/2).get())
    print("Bob:"+str(bobs_loss) + "Alice:"+str(alice_loss))

Exception ignored in: <bound method ObjectPointer.__del__ of [PointerTensor | me:54950316553 -> bob:45701892474]>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/syft/generic/pointers/object_pointer.py", line 220, in __del__
    messaging.ForceObjectDeleteMessage(self.id_at_location), self.location
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/base.py", line 236, in send_msg
    bin_response = self._send_msg(bin_message, location)
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/virtual.py", line 10, in _send_msg
    return location._recv_msg(message)
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/virtual.py", line 13, in _recv_msg
    return self.recv_msg(message)
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/base.py", line 270, in recv_msg
    response = self._message_router[msg_type](contents)
  File "/usr/local/lib/python3.6/dist-packages/syft/generic/object_storage.py", line 123, in force_rm_obj


RuntimeError: ignored

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

This even works for addition between two shared values!!

In [0]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [0]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

In [0]:
(bob_x_share + alice_x_share) % Q

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
# try this project here!
import random

In [0]:
x = 5
Q = 23740629843760239486723

In [0]:
def encrypt(x, n_shares = 3):
  
  shares = list()
  for i in range(n_shares-1):
    shares.append(random.randint(0, Q))
  final_share = Q- (sum(shares) % Q) + x
  shares.append(final_share)  
  return tuple(shares)

In [39]:
encrypt(5, n_shares=1)

(23740629843760239486728,)

In [0]:
def decrypt(shares):
  return sum(shares) % Q

In [0]:
def add(a,b):
  c = list()
  assert(len(a)== len(b))
  for i in range(len(a)):
    c.append((a[i]+b[i]) % Q)
  return tuple(c)

In [43]:
decrypt(add(encrypt(5), encrypt(10)))

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [49]:
encode(3.5)

35000

In [0]:
decode(35000)

In [0]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_worker)

In [0]:
bob._objects

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [0]:
y

In [0]:
y.get()

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [0]:
x

In [0]:
x = x.fix_prec()

In [0]:
x.child.child

In [0]:
y = x + x

In [0]:
y = y.float_prec()
y

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
y = x + x

In [0]:
y.get().float_prec()

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation